In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from upload_overleaf.upload import upload
import tensorflow as tf

In [ ]:
print(f'TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}')

The following cell reads the parquet files created by raw2parquet.py. 

It joins the prices and coacs tables on the ticker and date columns. It also creates a new column called OldNoOfStocks, which is the number of stocks before the stock split/dividend. If there is not a match in the coacs table, the OldNoOfStocks is set to 1. The StockOpen, StockHigh, StockLow, and StockClose columns are multiplied with OldNoOfStocks to get the adjusted price.

It applies the log transformation to the adjusted prices, and the calculates the log return for each ticker. The log return is calculated for the following time intervals: 1, 5, 10, 30, 60, 120, 240, and 390 minutes. 

In [ ]:
lf_intraday = (
    pl.scan_parquet('prices.parquet')
    .join(pl.scan_parquet('coacs.parquet').select(['ticker','date','OldNoOfStocks']), on=['ticker', 'date'], how='left')
    # If there is not a match in the coacs table, the OldNoOfStocks is set to 1.
    .with_columns(pl.when(pl.col('OldNoOfStocks').is_null()).then(1).otherwise(pl.col('OldNoOfStocks')).alias('OldNoOfStocks'))
    # Multiply StockOpen, StockHigh, StockLow, and StockClose with OldNoOfStocks to get the adjusted price
    .with_columns([
        pl.col('StockClose') * pl.col('OldNoOfStocks').alias('StockClose')
    ])
    .with_columns([
        pl.col('ticker'),
        pl.col('date').alias('date'),
        pl.col('datetime').alias('datetime'),
        pl.col('StockClose').log().alias('log_close'),
        pl.col('StockVol').alias('volume')
    ])
    .sort(['ticker', 'datetime'])
    .select([
        'ticker',
        'date',
        'datetime',
        'log_close',
        'volume',
        (pl.col('log_close') - pl.col('log_close').shift(1)).over(pl.col('ticker')).alias('return_1min'),
        (pl.col('log_close') - pl.col('log_close').shift(5)).over(pl.col('ticker')).alias('return_5min'),
        (pl.col('log_close') - pl.col('log_close').shift(10)).over(pl.col('ticker')).alias('return_10min'),
        (pl.col('log_close') - pl.col('log_close').shift(30)).over(pl.col('ticker')).alias('return_30min'),
        (pl.col('log_close') - pl.col('log_close').shift(60)).over(pl.col('ticker')).alias('return_1h'),
        (pl.col('log_close') - pl.col('log_close').shift(120)).over(pl.col('ticker')).alias('return_2h'),
        (pl.col('log_close') - pl.col('log_close').shift(240)).over(pl.col('ticker')).alias('return_4h'),
        (pl.col('log_close') - pl.col('log_close').shift(390)).over(pl.col('ticker')).alias('return_1d')
    ])
)

In [ ]:
lf_intraday.fetch()

In [ ]:
# Subset every 60th row. This is done to reduce the size of the data for initial testing.
lf_intraday = lf_intraday.gather_every(n=60).lazy()

The following cell transforms the intraday data to daily data. It groups the data by date and ticker, and sums the volume to get the daily volume. It also calculates the daily return for each ticker.

In [ ]:
# Group by date and ticker and sum volumne to get daily volume
lf_daily = (
    lf_intraday
    .group_by(['ticker', 'date'])
    .agg(
        pl.last('datetime').alias('datetime'),
        pl.last('log_close').cast(pl.Float64),
        pl.sum('volume').cast(pl.Float64).alias('volume') # Sum volume to get daily volume
    )
    .group_by(['ticker', 'date']).last() # Select the last row in each group
    .sort(['ticker', 'date'])
    .select([
        'ticker',
        'date',
        'datetime',
        'log_close',
        'volume',
        (pl.col('log_close') - pl.col('log_close').shift(1)).over(pl.col('ticker')).alias('return_1d')
    ])
)

In [ ]:
lf_daily.fetch(n_rows=10000)

In [ ]:
df_aapl = lf_daily.filter(pl.col('ticker') == 'AAPL').collect().to_pandas()

In [ ]:
# Plot time series of StockClose
plt.figure(figsize=(15,10))
# Add second y-axis. Left is log price and right is return
ax = sns.lineplot(x='date', y='log_close', data=df_aapl, color='cornflowerblue')
ax2 = ax.twinx()
sns.lineplot(x='date', y='volume', data=df_aapl, color='red', ax=ax2, alpha=0.5)
# Disable grid
ax.grid(False)
ax2.grid(False)
ax.set_title('AAPL Stock Price and Volume')
ax.set_ylabel('Log Price')
ax2.set_ylabel('Volume')
ax.set_xlabel('Date')
upload(plt, "Master's Thesis", 'figures/aapl_test.png')